In [1]:
import os
from glob import glob
import pandas as pd
import gzip
import zipfile
from IPython.display import display, HTML

In [2]:
def show_scrollable_df(df: pd.DataFrame, height: int = 400, width: int = 1000):
    display(HTML(
        df.to_html(notebook=True)
        .replace('<table border="1" class="dataframe">', 
                 f'<div style="height:{height}px; width:{width}px; overflow:auto; border:1px solid lightgray;"><table border="1" class="dataframe">')
        .replace('</table>', '</table></div>')
    ))


In [14]:
def list_zip_files(root_folder: str) -> list[str]:
    """
    Recursively list all .zip/.ZIP files under root_folder (year/month subfolders supported).
    """
    zips = []
    for dirpath, _, filenames in os.walk(os.path.expanduser(root_folder)):
        for fn in filenames:
            if fn.lower().endswith(".zip"):
                zips.append(os.path.join(dirpath, fn))
    zips.sort()
    return zips


In [4]:
def load_jsonl_gz(file_path: str) -> pd.DataFrame:
    try:
        with gzip.open(file_path, "rt", encoding="utf-8") as f:
            df = pd.read_json(f, lines=True)
        return df
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return pd.DataFrame()


In [17]:
def load_excel_zip(zip_path: str) -> pd.DataFrame:
    try:
        with zipfile.ZipFile(zip_path, "r") as z:
            # Find the BhavCopy file inside
            bhavcopy_file = [f for f in z.namelist() if f.startswith("BhavCopy_NCD_CO") and f.endswith(".xls")]
            if not bhavcopy_file:
                raise FileNotFoundError("No BhavCopy_NCD_CO...xls file found in the zip.")
            
            # Take the first match
            bhavcopy_file = bhavcopy_file[0]
            # print(bhavcopy_file)
            # Open and read directly with pandas
            with z.open(bhavcopy_file) as f:
                df = pd.read_excel(f)
        
        return df
    except Exception as e:
        print(f"Error reading from {zip_path}: {e}")
        return pd.DataFrame()

In [6]:
STATE_FIXES = {
    "orissa": "odisha",
    "pondicherry": "puducherry",
    "uttrakhand": "uttarakhand",
    "uttaranchal": "uttarakhand",
    "chattisgarh": "chhattisgarh",
    "jammu & kashmir": "jammu and kashmir",
    "delhi nct": "delhi",
    "nct of delhi": "delhi",
    "telengana": "telangana",
    "himanchal pradesh": "himachal pradesh",
    "andaman & nicobar islands": "andaman and nicobar islands",
    "dadra & nagar haveli": "dadra and nagar haveli and daman and diu",
    "daman & diu": "dadra and nagar haveli and daman and diu",
}

In [25]:
def collect_unique_symbols_from_zips(
    dir_path: str,
    column: str = "TckrSymb",
    normalize=True,
) -> list[str]:
    """
    Scan all .zip files in dir_path, read BhavCopy_NCD_CO... .xls/.xlsx inside,
    and return a sorted list of unique symbols from the given column.
    """
    uniq = set()
    zip_paths = list_zip_files(dir_path)
    for zip_path in zip_paths:
        # print(f"Processing {zip_path} ...")
        df = load_excel_zip(zip_path)
        if column not in df.columns:
            continue
        s = df[column].dropna().astype(str)
        if normalize:
            s = s.str.strip().str.upper()
        uniq.update(s.tolist())

    return sorted(uniq)

In [8]:
def normalize_state_column(df: pd.DataFrame, col: str, fixes: dict = None) -> None:
    """Normalize a state column in-place: strip, lowercase, unify spaces/& and apply fixes."""
    s = (
        df[col]
        .astype(str)
        .str.strip()
        .str.lower()                              # <- lowercase (the piece you wanted)
        .str.replace("&", " and ", regex=False)   # unify "&" -> "and"
        .str.replace(r"\s+", " ", regex=True)     # collapse multiple spaces
    )
    if fixes:
        s = s.replace(fixes)
    df[col] = s

In [18]:
enam_file = "../data/enam/2025/Aug/2025-08-01.jsonl.gz"
agmarknet_file = "../data/agmarknet/2025/Aug/2025-08-01.jsonl.gz"
ncdex_file = "../data/ncdex/2025/Aug/2025-08-01.zip"

In [19]:
df1 = load_jsonl_gz(enam_file)
df2 = load_jsonl_gz(agmarknet_file)
df3 = load_excel_zip(ncdex_file)
col1 = "state"
col2 = "state_name"
normalize_state_column(df1, col1, STATE_FIXES)
normalize_state_column(df2, col2, STATE_FIXES)

In [20]:
counts1 = df1['state'].value_counts().rename("count_df1")
counts2 = df2['state_name'].value_counts().rename("count_df2")

# Outer join so all states from both appear
result = pd.concat([counts1, counts2], axis=1).fillna(0).astype(int)

# Optional: add difference column
result["diff"] = result["count_df1"] - result["count_df2"]

In [21]:
result

,count_df1,count_df2,diff
haryana,544,648,-104
rajasthan,342,556,-214
uttar pradesh,237,2828,-2591
odisha,156,314,-158
uttarakhand,116,180,-64
gujarat,108,944,-836
maharashtra,87,1258,-1171
madhya pradesh,85,1502,-1417
tamil nadu,81,6162,-6081
himachal pradesh,81,519,-438


In [22]:
print(df3.shape)
show_scrollable_df(df3, height=500, width=1200)


(1177, 34)


,TradDt,BizDt,Sgmt,Src,FinInstrmTp,FinInstrmId,ISIN,TckrSymb,SctySrs,XpryDt,FininstrmActlXpryDt,StrkPric,OptnTp,FinInstrmNm,OpnPric,HghPric,LwPric,ClsPric,LastPric,PrvsClsgPric,UndrlygPric,SttlmPric,OpnIntrst,ChngInOpnIntrst,TtlTradgVol,TtlTrfVal,TtlNbOfTxsExctd,SsnId,NewBrdLotQty,Rmks,Rsvd1,Rsvd2,Rsvd3,Rsvd4
0,2025-08-01,2025-08-01,CO,NCD,COF,75678,NaN,BAJRA,NaN,2025-08-20,2025-08-20,NaN,NaN,BAJRA20AUG2025,0.0,0.0,0.0,2388.0,0.0,2398.0,2380.00,2388.0,0,0,0,0,0,F1,10,NaN,NaN,NaN,NaN,NaN
1,2025-08-01,2025-08-01,CO,NCD,COF,76176,NaN,BAJRA,NaN,2025-09-19,2025-09-19,NaN,NaN,BAJRA19SEP2025,0.0,0.0,0.0,2400.0,0.0,2410.0,2380.00,2400.0,0,0,0,0,0,F1,10,NaN,NaN,NaN,NaN,NaN
2,2025-08-01,2025-08-01,CO,NCD,COF,76592,NaN,BAJRA,NaN,2025-10-20,2025-10-20,NaN,NaN,BAJRA20OCT2025,0.0,0.0,0.0,2412.0,0.0,2423.0,2380.00,2412.0,0,0,0,0,0,F1,10,NaN,NaN,NaN,NaN,NaN
3,2025-08-01,2025-08-01,CO,NCD,COF,77041,NaN,BAJRA,NaN,2025-11-20,2025-11-20,NaN,NaN,BAJRA20NOV2025,0.0,0.0,0.0,2424.0,0.0,2435.0,2380.00,2424.0,0,0,0,0,0,F1,10,NaN,NaN,NaN,NaN,NaN
4,2025-08-01,2025-08-01,CO,NCD,COF,77429,NaN,BAJRA,NaN,2025-12-19,2025-12-19,NaN,NaN,BAJRA19DEC2025,0.0,0.0,0.0,2436.0,0.0,2435.0,2380.00,2436.0,0,0,0,0,0,F1,10,NaN,NaN,NaN,NaN,NaN
5,2025-08-01,2025-08-01,CO,NCD,COF,75679,NaN,BARLEYJPR,NaN,2025-08-20,2025-08-20,NaN,NaN,BARLEYJPR20AUG2025,0.0,0.0,0.0,2337.5,0.0,2345.5,2330.20,2337.5,0,0,0,0,0,F1,10,NaN,NaN,NaN,NaN,NaN
6,2025-08-01,2025-08-01,CO,NCD,COF,76177,NaN,BARLEYJPR,NaN,2025-09-19,2025-09-19,NaN,NaN,BARLEYJPR19SEP2025,0.0,0.0,0.0,2349.5,0.0,2357.0,2330.20,2349.5,0,0,0,0,0,F1,10,NaN,NaN,NaN,NaN,NaN
7,2025-08-01,2025-08-01,CO,NCD,COF,76593,NaN,BARLEYJPR,NaN,2025-10-20,2025-10-20,NaN,NaN,BARLEYJPR20OCT2025,0.0,0.0,0.0,2361.0,0.0,2369.0,2330.20,2361.0,0,0,0,0,0,F1,10,NaN,NaN,NaN,NaN,NaN
8,2025-08-01,2025-08-01,CO,NCD,COF,77042,NaN,BARLEYJPR,NaN,2025-11-20,2025-11-20,NaN,NaN,BARLEYJPR20NOV2025,0.0,0.0,0.0,2373.5,0.0,2381.0,2330.20,2373.5,0,0,0,0,0,F1,10,NaN,NaN,NaN,NaN,NaN
9,2025-08-01,2025-08-01,CO,NCD,COF,77430,NaN,BARLEYJPR,NaN,2025-12-19,2025-12-19,NaN,NaN,BARLEYJPR19DEC2025,0.0,0.0,0.0,2384.5,0.0,2381.0,2330.20,2384.5,0,0,0,0,0,F1,10,NaN,NaN,NaN,NaN,NaN


In [26]:
unique_symbols = collect_unique_symbols_from_zips("../data/ncdex")
unique_symbols

Error reading from ../data/ncdex/2025/Apr/2025-04-05.zip: File is not a zip file
Error reading from ../data/ncdex/2025/Apr/2025-04-06.zip: File is not a zip file
Error reading from ../data/ncdex/2025/Apr/2025-04-10.zip: File is not a zip file
Error reading from ../data/ncdex/2025/Apr/2025-04-12.zip: File is not a zip file
Error reading from ../data/ncdex/2025/Apr/2025-04-13.zip: File is not a zip file
Error reading from ../data/ncdex/2025/Apr/2025-04-14.zip: File is not a zip file
Error reading from ../data/ncdex/2025/Apr/2025-04-18.zip: File is not a zip file
Error reading from ../data/ncdex/2025/Apr/2025-04-19.zip: File is not a zip file
Error reading from ../data/ncdex/2025/Apr/2025-04-20.zip: File is not a zip file
Error reading from ../data/ncdex/2025/Apr/2025-04-26.zip: File is not a zip file
Error reading from ../data/ncdex/2025/Apr/2025-04-27.zip: File is not a zip file
Error reading from ../data/ncdex/2025/Aug/2025-08-02.zip: File is not a zip file
Error reading from ../data/n

['BAJRA',
 'BARLEYJPR',
 'CASTOR',
 'CASTOROIL',
 'COCUDAKL',
 'COTTON',
 'COTWASOIL',
 'DHANIYA',
 'GROUNDNUT',
 'GUARGUM5',
 'GUARSEED10',
 'ISABGOL',
 'JEERAMINI',
 'JEERAUNJHA',
 'KAPAS',
 'MAIZE',
 'SESAMESEED',
 'STEEL',
 'SUNOIL',
 'TMCFGRNZM',
 'YELLOWP']

In [12]:
glob(os.path.join("../data/ncdex", "*.zip"),recursive=True)

[]